# Установка библиотек

In [1]:
import pandas as pd #для работы с таблицами

Для автоматического применения статистического критерия (теста) используется модуль `stats` библиотеки `scipy`.

Этот модуль содержит большое количество вероятностных распределений, корреляционные функции и статистические тесты и многое другое.

Также активно используется библиотека `statsmodels`

Для установки библиотеки `scipy` и `statsmodels` можно воспользоваться менеджером пакетов `pip`.

In [1]:
#!pip install scipy

In [2]:
#!pip install statsmodels

iiii# Загрузка данных 

Для скачивания данных используем команду `!wget`. 

Данные появятся в локальной директории.

Мы будем использовать информацию о диаметре пицц разных производителей.

In [6]:
!wget https://raw.githubusercontent.com/harika-bonthu/Hypothesis-test-examples/main/pizzas.csv

--2022-04-16 17:33:57--  https://raw.githubusercontent.com/harika-bonthu/Hypothesis-test-examples/main/pizzas.csv
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 539 [text/plain]
Сохранение в: «pizzas.csv.1»

pizzas.csv.1        100%[===================>]     539  --.-KB/s    за 0s      

2022-04-16 17:33:58 (8,73 MB/s) - «pizzas.csv.1» сохранён [539/539]



Загрузим данные в память ноутбука.

In [7]:
data = pd.read_csv('pizzas.csv')

In [8]:
data.rename(columns={'Making Unit 1': 'Пиццерия №1', 'Making Unit 2': 'Пиццерия №2'}, inplace=True)

In [9]:
data.head()

,Пиццерия №1,Пиццерия №2
0,6.8090,6.7703
1,6.4376,7.5093
2,6.9157,6.7300
3,7.3012,6.7878
4,7.4488,7.1522


# Проверка данных на нормальность


In [10]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

Установим уровень значимости

In [11]:
alpha = 0.05

## Тест Шапиро-Уилка

In [12]:
from scipy.stats import shapiro

In [13]:
_, p = shapiro(data)
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.204
Данные распределены нормально


In [14]:
shapiro(data)

ShapiroResult(statistic=0.9762489795684814, pvalue=0.2044535130262375)

## Тест Д’Агостино

In [15]:
from scipy.stats import normaltest

In [16]:
_, p = normaltest(data)
#normaltest возвращает двустороннюю вероятность для проверки гипотезы

print('p=%.3f' % p[0])

# Интерпретация 

if p[0] > alpha/2:
	print(H0)
else:
	print(Ha)
 
if p[1] > alpha/2:
	print(H0)
else:
	print(Ha)

p=0.251
Данные распределены нормально
Данные распределены нормально


In [17]:
normaltest(data['Пиццерия №1'])

NormaltestResult(statistic=2.761164477860819, pvalue=0.25143211686667205)

In [18]:

normaltest(data['Пиццерия №2'])

NormaltestResult(statistic=1.228479990932946, pvalue=0.541051941019278)

In [19]:
normaltest(data)

NormaltestResult(statistic=array([2.76116448, 1.22847999]), pvalue=array([0.25143212, 0.54105194]))

# Независимый T-тест 

In [20]:
from scipy.stats import ttest_ind

In [21]:
H0 = 'Нет значимой разницы между диаметрами пиццы в разных пиццериях.'
Ha = 'Есть значимая разница между диаметрами пиццы в разных пиццериях.'

Зависимая переменная (диаметр пиццы) является количественной. Группы происходят из разных совокупностей. Следовательно, мы используем независимый T-тест.

In [22]:
def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(data['Пиццерия №1'], data['Пиццерия №2'], equal_var=True)

    p = round(test_results[1],2)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(data)


*** Результаты независимого T-теста ***
0.47 > 0.05. Мы не можем отвергнуть нулевую гипотезу. Нет значимой разницы между диаметрами пиццы в разных пиццериях.


In [23]:
test_results = ttest_ind(data['Пиццерия №1'], data['Пиццерия №2'], equal_var=True)
test_results

Ttest_indResult(statistic=0.7228688704678063, pvalue=0.4722394724599501)

---

# Корреляция Спирмана 

In [24]:
from numpy.random import rand
from scipy.stats import spearmanr

data1 = rand(1000) * 20
data2 = data1 + (rand(1000) * 10)


corr, p = spearmanr(data1, data2)

In [25]:
print(corr,p)

0.8984414024414025 0.0


In [26]:
if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.")

0.0 <= 0.05. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.


# ANOVA тест

In [27]:
from scipy.stats import f_oneway

Однофакторный дисперсионный анализ ANOVA проверяет нулевую гипотезу о том, что две или более групп имеют одинаковое среднее значение генеральной совокупности.

В качестве данных возьмем информацию по размерам раковины мидий, выращенных в разных местах.

In [28]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]
tvarminne = [0.0703, 0.1026, 0.0956, 0.0973, 0.1039, 0.1045]

In [29]:
_, p = f_oneway(petersburg, magadan, tvarminne)

In [30]:
H0 = 'Нет значимой разницы между средним размером раковины мидий в трех разных местах.'
Ha = 'Есть значимая разница между средним размером раковины мидий в трех разных местах.'

In [31]:
if p>alpha:
  print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
  print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

0.007870900547143829 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между средним размером раковины мидий в трех разных местах.


_____

# Домашнее задание

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.
 
 
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.


#### Объедините два массива в DataFrame

In [32]:
len(petersburg)

7

In [33]:
len(magadan)

8

In [34]:
petersburg.append(pd.Series(petersburg).median())

In [35]:
df=pd.DataFrame({
    'petersburg': petersburg,
    'magadan': magadan})

#### Оценка нормальности распределения

In [36]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'
#Уровень значимости
alpha=0.05

In [37]:
for col in df.columns:
    _,p=shapiro(df[col])
   # print('p=%.3f' % p)
    print("%a p=%.2f" % (col,p))
    if p>alpha:
        print(H0)
    else:
        print(Ha)

'petersburg' p=0.11
Данные распределены нормально
'magadan' p=0.04
Данные не распределены нормально (мы отвергаем H0)


In [38]:
_,p=normaltest(df)
i=0
for col in df.columns:
    print(col)
    if p[i]>alpha/2:
        print(H0);
    else:
        print(Ha);

petersburg
Данные распределены нормально
magadan
Данные распределены нормально


/home/elena/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


Найдем коэфициент корреляции и проверим гипотезу Н0-между данными нет корреляции.
Поскольку данные непрерывные и распределены нормально, то используеи корреляционным тестом Пирсона:


In [39]:
H0='между данными нет корреляции'
Ha='между данными есть корреляция'
from scipy.stats import pearsonr
test_p=pearsonr(df['petersburg'], df['magadan'])
print(f'Коэфициент корреляции={test_p[0]}')
if test_p[1]>alpha/2:
    print(f"P_value={test_p[1]} > {alpha/2}. Мы не можем отвергнуть нулевую гипотезу: {H0}")
else:
    print(f"P_value={test_p[1]} <= {alpha/2}. Мы отвергаем нулевую гипотезу: {Ha}")


Коэфициент корреляции=0.2566651511050131
P_value=0.5394705530383969 > 0.025. Мы не можем отвергнуть нулевую гипотезу: между данными нет корреляции


Найдем коэфициент корреляции и проверим гипотезу Н0-между данными нет корреляции.
Поскольку данные количественные и распределены ненормально, то воспользуемся корреляционным тестом Спирмана:

In [40]:
df.corr(method='spearman')

,petersburg,magadan
petersburg,1.000000,0.155691
magadan,0.155691,1.000000


In [41]:
H0='между данными нет корреляции'
Ha='между данными есть корреляция'
from scipy.stats import spearmanr
test_s=spearmanr(df['petersburg'], df['magadan'])
print(f'Коэфициент корреляции={test_s[0]}')
if test_p[1]>alpha:
    print(f"P_value={test_s[1]} > {alpha}. Мы не можем отвергнуть нулевую гипотезу: {H0}")
else:
    print(f"P_value={test_s[1]} <= {alpha}. Мы отвергаем нулевую гипотезу: {Ha}")


Коэфициент корреляции=0.15569141404872366
P_value=0.7127617079991629 > 0.05. Мы не можем отвергнуть нулевую гипотезу: между данными нет корреляции


In [42]:
test_s

SpearmanrResult(correlation=0.15569141404872366, pvalue=0.7127617079991629)

Вывод:   между данными очень слабая связь

Зависимая переменная (размер мидий) является количественной и распределены нормально,группы происходят из разных совокупностей. Следовательно, мы используем независимый T-тест.

In [43]:
H0 = 'Выборки имеют одинаковые средние значения'
Ha = 'Есть значимая разница между диаметрами пиццы в разных пиццериях.'
test_result=ttest_ind(df['petersburg'], df['magadan'])
p = round(test_results[1],2)
if p>alpha:
    print(f"P-value={p} > {alpha/2}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
    print(f"P-value={p} <= {alpha/2}. Мы отвергаем нулевую гипотезу: {Ha}")


P-value=0.47 > 0.025. Мы не можем отвергнуть нулевую гипотезу. Выборки имеют одинаковые средние значения


In [44]:
test_result

Ttest_indResult(statistic=3.5945040471515983, pvalue=0.0029302645160802256)

In [45]:
m=df.mean()
(m[0]+m[1])/2

0.0906125

In [ ]:
df['mean']=df.petersburg-df.magadan

In [ ]:
df.iloc[:, 2].sum()

0.20159999999999995

Статистически значимая связь между размерами мидий отсутствует

# Бонус 10



Скачаем датасет, содержащий информацию по давлению 120 человек.



In [ ]:
!wget https://raw.githubusercontent.com/yug95/MachineLearning/master/Hypothesis%20testing/blood_pressure.csv

--2022-04-14 17:40:05--  https://raw.githubusercontent.com/yug95/MachineLearning/master/Hypothesis%20testing/blood_pressure.csv
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 2851 (2,8K) [text/plain]
Сохранение в: «blood_pressure.csv»

blood_pressure.csv  100%[===================>]   2,78K  --.-KB/s    за 0s      

2022-04-14 17:40:05 (7,00 MB/s) - «blood_pressure.csv» сохранён [2851/2851]



In [ ]:
data = pd.read_csv('blood_pressure.csv')

In [ ]:
print('Размер выборки: ', data.shape)

Размер выборки:  (120, 5)


In [ ]:
data.head()

,patient,sex,agegrp,bp_before,bp_after
0,1,Male,30-45,143,153
1,2,Male,30-45,163,170
2,3,Male,30-45,153,168
3,4,Male,30-45,153,142
4,5,Male,30-45,146,141


## Z-тест

Проведем Z-тест для проверки двух независимых групп данных на равенство средних значений выборок.

In [ ]:
from statsmodels.stats import weightstats

In [ ]:
_ ,p = weightstats.ztest(data['bp_before'], x2=data['bp_after'], value=0,alternative='two-sided')
print(float(p))


if p < 0.05:
    print(f"Мы отвергаем нулевую гипотезу об отсутсвии различий между данными.p={p}")
else:
    print("Мы не можем отвергнуть нулевую гипотезу об отсутсвии различий между данными.")

0.002162306611369422
Мы отвергаем нулевую гипотезу об отсутсвии различий между данными.p=0.002162306611369422


## Z-тест на пропорцию

Z-критерий одной пропорции используется для сравнения наблюдаемой пропорции с теоретической.

В этом тесте используются следующие нулевые гипотезы:

$H_0: p = p_0$ (доля мужчин, предоставивших данные о своем давлении, равна гипотетической пропорции $p_0$)

Альтернативная гипотеза может быть двусторонней, левосторонней или правосторонней:

$H_1 (двусторонний): p ≠ p_0$ (доля мужчин не равна некоторому гипотетическому значению $p_0$)
$H_1 (левосторонний): p < p_0$ (доля мужчин меньше некоторого гипотетического значения $p_0$)
$H_1 (правосторонний): p > p_0$ (доля мужчин больше некоторого гипотетического значения $p_0$)

Предположим, что доля мужчин в нашем датачете равна 40%.

$p_0$: гипотетическая доля мужчин = 0.40

$x$: число мужчин в выборке мужчин: `len(data[data.sex == 'Male'])`

$n$: размер выборки = `len(data)`

Покажем, как использовать функцию `proportions_ztest` для выполнения  **z-теста**:

In [ ]:
p_0 = 0.41
n = len(data)
x = len(data[data.sex == 'Male'])

print(n, x)

120 60


In [ ]:
from statsmodels.stats.proportion import proportions_ztest

#perform one proportion z-test
_, p = proportions_ztest(count=x, nobs=n, value=p_0)

In [ ]:
if p < 0.05:
    print(f"Мы отвергаем нулевую гипотезу о том, что доля мужчин, предоставивших данные о своем давлении, равна 40%.p={p}")
else:
    print(f"Мы не можем отвергнуть нулевую гипотезу.p={p}")

Мы отвергаем нулевую гипотезу о том, что доля мужчин, предоставивших данные о своем давлении, равна 40%.p=0.04863230451469028
